#### 4-1) train_data.csv 파일의 학습데이터 100 개를 이용하여 decision tree (결정 트리)를 학습하시오

In [1]:
import pandas as pd 
import numpy as np

pd_data = pd.read_csv('./train_data.csv') # train_data.csv 파일 읽어 오기

features = pd_data.iloc[:100, :-1] # 100개의 데이터만 선택,  features 변수에 할당
target = pd_data.iloc[:100, -1:] # # 100개의 레이블만 선택, target 변수에 할당

def entropy(y): # 엔트로피 계산 함수 정의
    # y에 있는 클래스 레이블들의 엔트로피를 계산합니다.
    cnt = np.unique(y, return_counts=True)[1]
    probs = cnt / len(y)
    entropy = -np.sum(probs * np.log2(probs))
    return entropy

def information_gain(X, y, feature_idx): # 정보 이득 계산 함수 정의
    # feature_idx에 해당하는 특징을 사용하여 분할했을 때의 정보 이득을 계산합니다.
    p_entropy = entropy(y)
    values, cnt = np.unique(X.iloc[:, feature_idx], return_counts=True)
    c_entropy = 0
    for value, cnt in zip(values, cnt):
        child_y = y[X.iloc[:, feature_idx] == value]
        c_entropy += cnt / len(X) * entropy(child_y)
    ig = p_entropy - c_entropy
    return ig

def find_best_feature(X, y): # 가장 정보 이득이 큰 특징을 찾는 함수 정의
    # 모든 특징들 중에서 가장 정보 이득이 큰 특징을 찾아 반환합니다.
    best_feature = None
    best_ig = 0
    for i in range(X.shape[1]):
        ig = information_gain(X, y, i)
        if ig > best_ig:
            best_feature = i
            best_ig = ig
    return best_feature

def build_tree(X, y): # Decision tree를 구성하는 함수 정의
    # 재귀적으로 decision tree를 구성합니다.
    # Base case: 모든 레이블이 동일한 경우
    if len(np.unique(y)) == 1:
        return y.iloc[0]
    # 먼저 base case를 처리하고, 더 이상 분할할 특징이 없을 때, 데이터의 가장 많은 레이블을 반환
    if X.shape[1] == 0:
        return np.bincount(y).argmax()
    # recursive case에서는 find_best_feature 함수를 사용하여 가장 정보 이득이 큰 특징을 찾고, 그 특징의 값을 기준으로 데이터를 분할합니다.
    best_feature = find_best_feature(X, y)
    tree = {best_feature: {}}
    values = np.unique(X.iloc[:, best_feature])
    for val in values:
        index = X.iloc[:, best_feature] == val
        subtree = build_tree(X.loc[index], y.loc[index])
        tree[best_feature][val] = subtree
    return tree

tree = build_tree(features, target) # 트리 만들기
print(tree) # 학습된 Decision tree를 출력

{0: {0: {3: {0: {1: {0: class    Cat
Name: 97, dtype: object, 1: class    Dog
Name: 48, dtype: object}}, 1: class    Cat
Name: 50, dtype: object}}, 1: {2: {0: class    Dog
Name: 0, dtype: object, 1: {1: {0: class    Cat
Name: 86, dtype: object, 1: class    Dog
Name: 45, dtype: object}}}}}}


#### 4-2) test_data.csv 파일의 테스트 데이터 10 개에 대한 테스트 결과를 출력하시오.

In [2]:
import pandas as pd # pandas 불러오기

pd_data = pd.read_csv('./test_data.csv') # test_data.csv 파일 읽어오기

for i in range(10): # 10번 반복 실행
    row = pd_data.iloc[i, :] # i번째 데이터 선택
    x = row[:-1] # feature 값
    true_label = row[-1] # true label 값
    node = tree # decision tree에서 루트 노드를 시작으로 시작
    while isinstance(node, dict): # 현재 노드가 dictionary(내부 노드)인 경우, 다음 노드로 이동
        feature = list(node.keys())[0] # 현재 노드의 feature 값
        value = x[feature] # feature 값에 해당하는 데이터 
        node = node[feature][value] # 다음 노드로 이동
    pred_label = node # leaf node에 도달하면, 해당 노드에서 반환하는 값을 예측값으로 저장
    print(f"Test #{i} {list(x)} -> {pred_label}") # 출력

Test #0 [1, 1, 0, 0] -> class    Dog
Name: 0, dtype: object
Test #1 [0, 0, 1, 1] -> class    Cat
Name: 50, dtype: object
Test #2 [1, 1, 0, 0] -> class    Dog
Name: 0, dtype: object
Test #3 [0, 0, 1, 1] -> class    Cat
Name: 50, dtype: object
Test #4 [1, 1, 0, 1] -> class    Dog
Name: 0, dtype: object
Test #5 [1, 1, 1, 0] -> class    Dog
Name: 45, dtype: object
Test #6 [1, 0, 1, 1] -> class    Cat
Name: 86, dtype: object
Test #7 [1, 0, 0, 1] -> class    Dog
Name: 0, dtype: object
Test #8 [0, 1, 1, 1] -> class    Cat
Name: 50, dtype: object
Test #9 [1, 1, 1, 0] -> class    Dog
Name: 45, dtype: object
